In [31]:
import random

class World:
    def createWorld(this, dirt_count):
        # grid size (10x10)
        this.n = 10                 

        # Create grid filled with "c"
        this.map = [["c" for _ in range(this.n)] for _ in range(this.n)]
        
        # Add walls "l"
        for i in range(this.n):
            this.map[0][i] = "l"
            this.map[this.n-1][i] = "l"
            this.map[i][0] = "l"
            this.map[i][this.n-1] = "l"
        
        # Add obstacles "#"
        this.map[3][3] = "#"
        this.map[3][4] = "#"
        this.map[7][6] = "#"
        
        # Add random dirt "*"
        this.dirt = 0
        while this.dirt < dirt_count:
            r = random.randint(1, this.n-2)
            c = random.randint(1, this.n-2)
            if this.map[r][c] == "c":
                this.map[r][c] = "*"
                
                this.dirt += 1

    def show(this, agent):
        for r in range(this.n):
            display_row = this.map[r][:]
            if r == agent.row:
                display_row[agent.col] = "A"
            print(" ".join(display_row))

class Agent:
    
    def __init__(this):#Constructor method for the agent
        this.row = 1
        this.col = 1       
        
        # Performance counters
        this.moves     = 0
        this.cleans    = 0
        this.wall_hits = 0

    # Sensors
    def isDirty(this, world):
        return world.map[this.row][this.col] == "*"
        
    # Actions 
    def moveRight(this, world):
        if world.map[this.row][this.col + 1] not in ["l", "#"]:
            this.col += 1
            this.moves += 1
            return True
        this.wall_hits += 1
        return False

    def moveLeft(this, world):
        if world.map[this.row][this.col - 1] not in ["l", "#"]:
            this.col -= 1
            this.moves += 1
            return True
        this.wall_hits += 1
        return False

    def moveUp(this, world):
        if world.map[this.row - 1][this.col] not in ["l", "#"]:
            this.row -= 1
            this.moves += 1
            return True
        this.wall_hits += 1
        return False

    def moveDown(this, world):
        if world.map[this.row + 1][this.col] not in ["l", "#"]:
            this.row += 1
            this.moves += 1
            return True
        this.wall_hits += 1
        return False

    def suck(this, world):
        world.map[this.row][this.col] = "c"
        world.dirt -= 1
        this.cleans += 1 

    def start(this): #resets the location to first cell, and it resets the perfoemance counters to 0
        # Agent start position
        this.row = 1
        this.col = 1       
        
        # Performance counters
        this.moves     = 0
        this.cleans    = 0
        this.wall_hits = 0

# program test
world = World()
world.createWorld(10) #providing the number of dirty cells for the world

agent = Agent()

world.show(agent)

l l l l l l l l l l
l A * c c c c c * l
l c c c c c * c c l
l * c # # * c c c l
l * * * c c c c c l
l c c c c c c c c l
l c c * c c c c c l
l c c c c c # c c l
l c c c * c c c c l
l l l l l l l l l l


In [109]:
def randomMove(agent): #chooses a random direction other than right to move
    possibleMoves = ["Right", "Left", "Up", "Down"]     
    
    while(len(possibleMoves) > 0): #loop until the agent selects a direction that is not blocked or len(possibleMoves) = 0
        nextMove = random.choice(possibleMoves) #random selector
        if(nextMove == "Right"):
            if(agent.moveRight(world)):
                break #agent moved successfully
            else:
                possibleMoves.remove("Right") #to minimize the number of loops
        if(nextMove == "Left"):
            if(agent.moveLeft(world)):
                break #agent moved successfully
            else:
                possibleMoves.remove("Left") #to minimize the number of loops
        elif(nextMove == "Up"):
            if(agent.moveUp(world)):
                break #agent moved successfully
            else:
                possibleMoves.remove("Up") #to minimize the number of loops
        elif(nextMove == "Down"):
            if(agent.moveDown(world)):
                break #agent moved successfully
            else:
                possibleMoves.remove("Down") #to minimize the number of loops

def simpleReflexAgent(agent):
    visited = set() #monitring visited locations to avoid looping
    while(world.dirt > 0):
        location = (agent.row, agent.col) #make a location as a set
        if location not in visited: #if it is a new location
            visited.add(location)
            if(agent.isDirty(world)):
                agent.suck(world)
                
            if(agent.moveRight(world)): #checks if moving right is possible and successfully done
                continue 

        randomMove(agent) #chooses a random direction if moving right is not possible


world.createWorld(10)
agent.start()
print("Room before Cleaning Room")
world.show(agent)

simpleReflexAgent(agent)

print("\nRoom after Cleaning")
world.show(agent)

Room before Cleaning Room
l l l l l l l l l l
l A c * c c c c * l
l c c c c c c c c l
l * c # # c * c c l
l c c c c c c c * l
l c c c c c c c c l
l c c c * * c c c l
l c c * c * # c c l
l c c c c * c c c l
l l l l l l l l l l

Room after Cleaning
l l l l l l l l l l
l c c c c c c c c l
l c c c c c c c c l
l c A # # c c c c l
l c c c c c c c c l
l c c c c c c c c l
l c c c c c c c c l
l c c c c c # c c l
l c c c c c c c c l
l l l l l l l l l l


In [110]:
def createDirectionTable():
    
    table = []
    
    # Create snake pattern for 8x8 inner grid
    for row in range(8):  
        if row % 2 == 0:  
            for _ in range(7):  # Move right 7 times
                table.append("Right")

        else:  # Odd rows: move left
            for _ in range(7):  # Move left 7 times
                table.append("Left")
        
        # Move down after each row 
        if row < 7:
            table.append("Down")
    
    return table


def tryAlternativeMove(agent):
   
    alternatives = ["Right", "Left", "Up", "Down"]
    random.shuffle(alternatives)
    
    for direction in alternatives:
        if direction == "Right" and agent.moveRight(world):
            return True
        elif direction == "Left" and agent.moveLeft(world):
            return True
        elif direction == "Up" and agent.moveUp(world):
            return True
        elif direction == "Down" and agent.moveDown(world):
            return True
    
    return False


def tableDrivenAgent(agent):
    #Table Driven Agent: Uses a predefined lookup table for navigation.

    # The direction table
    directionTable = createDirectionTable()
    tableIndex = 0
    maxIterations = 500  
    iterations = 0
    
    while world.dirt > 0 and iterations < maxIterations:
        iterations += 1
        
        # Check and clean if current cell is dirty
        if agent.isDirty(world):
            agent.suck(world)
        
        # Get next direction from table
        if tableIndex < len(directionTable):
            nextDirection = directionTable[tableIndex]
            tableIndex += 1
            
            # Try to move according to table
            moved = False
            if nextDirection == "Right":
                moved = agent.moveRight(world)
            elif nextDirection == "Left":
                moved = agent.moveLeft(world)
            elif nextDirection == "Up":
                moved = agent.moveUp(world)
            elif nextDirection == "Down":
                moved = agent.moveDown(world)
            
            # If table direction failed , try alternatives
            if not moved:
                tryAlternativeMove(agent)
        else:
            #search for remaining dirt randomly
            tryAlternativeMove(agent)
    
    if iterations >= maxIterations:
        print("Reached maximum iterations")


# Test the Table Driven Agent

# Create a new world for testing
world.createWorld(10)
agent.start()

print("Room before Cleaning Room")
world.show(agent)

tableDrivenAgent(agent)

print("\nRoom after Cleaning")
world.show(agent)

Room before Cleaning Room
l l l l l l l l l l
l A c c c c c c c l
l c c c * c c c c l
l c c # # c c c c l
l c c c * c c c * l
l c c c c * c c c l
l c c c c c * c c l
l c c c c c # * c l
l c c c * c * c * l
l l l l l l l l l l

Room after Cleaning
l l l l l l l l l l
l c c c c c c c c l
l c c c c c c c c l
l c c # # c c c A l
l c c c c c c c c l
l c c c c c c c c l
l c c c c c c c c l
l c c c c c # c c l
l c c c c c c c c l
l l l l l l l l l l


In [113]:
def calculate_score(agent):
    # lower score = better performance
    return agent.moves + agent.wall_hits

def getScore(agent):
    print(f"  Total Moves   : {agent.moves}")
    print(f"  Cells Cleaned : {agent.cleans}")
    print(f"  Wall Hits     : {agent.wall_hits}")
    print(f"  Score         : {calculate_score(agent)}")
    return calculate_score(agent)

# Test Simple Reflex Agent
world.createWorld(10)
agent1 = Agent()
print("\nBefore cleaning:\n")
world.show(agent1)

simpleReflexAgent(agent1)

print("\nAfter cleaning:\n")
world.show(agent1)

print("\nSimple Reflex Agent Performance:\n")
score1 = getScore(agent1)

print("\n=============================================================================\n")


# Test Table Driven Agent
world.createWorld(10)
agent2 = Agent()
print("\nBefore cleaning:\n")
world.show(agent2)

tableDrivenAgent(agent2)

print("\nAfter cleaning:\n")
world.show(agent2)

print("\nTable Driven Agent Performance:\n")
score2 = getScore(agent2)

print("\n=============================================================================\n")
# Comparison
print("Simple Reflex Agent  vs  Table Driven Agent")
print(f"Total Moves   : {agent1.moves}  vs  {agent2.moves}")
print(f"Cells Cleaned : {agent1.cleans}  vs  {agent2.cleans}")
print(f"Wall Hits     : {agent1.wall_hits}  vs  {agent2.wall_hits}")
print(f"Score         : {score1}  vs  {score2}\n")

if score1 < score2:
    print("Simple Reflex Agent is better")
elif score2 < score1:
    print("Table Driven Agent is better")
else:
    print("Both agents performed equally")


Before cleaning:

l l l l l l l l l l
l A c c c c c c c l
l * c c c c c * c l
l c c # # c c c * l
l c c * c * c * c l
l c c c c c * c c l
l c c c c c c c c l
l c c * c c # c * l
l c * c c c c c c l
l l l l l l l l l l

After cleaning:

l l l l l l l l l l
l c c c c c c c c l
l c c c c c c c c l
l c c # # c c c c l
l c c c c c c c c l
l c c c c c c c c l
l c c c c c c c c l
l c c c c c # c c l
l c c A c c c c c l
l l l l l l l l l l

Simple Reflex Agent Performance:

  Total Moves   : 465
  Cells Cleaned : 10
  Wall Hits     : 71
  Score         : 536



Before cleaning:

l l l l l l l l l l
l A c c c c c c c l
l * c c c * c c * l
l c c # # c * * c l
l c c c c c * c c l
l c c c c c c c c l
l * c c c c * c c l
l * c c c * # c c l
l c c c c c c c c l
l l l l l l l l l l

After cleaning:

l l l l l l l l l l
l c c c c c c c c l
l c c c c c c c c l
l c c # # c c c c l
l c c c c c c c c l
l c c c c c c c c l
l c c c c c c c c l
l c c c A c # c c l
l c c c c c c c c l
l l l l l l l l l l

Ta